# Usando _monkey patch_ pra implementar `coalesce` para o DataFrame do pandas

## Como se faz `coalesce` atualmente

In [1]:
import pandas as pd
nan = pd.np.nan

In [2]:
df = pd.DataFrame([[1,   2,   3],
                   [nan, 2,   3],
                   [nan, nan, 3],
                   [1,   nan, 3],
                   [nan, nan, nan]],
                  columns=['one', 'two', 'three'])
df

,one,two,three
0,1,2,3
1,NaN,2,3
2,NaN,NaN,3
3,1,NaN,3
4,NaN,NaN,NaN


Usando `fillna` (possivelmente vários deles)

In [3]:
df.one.fillna(df.two).fillna(df.three)

0     1
1     2
2     3
3     1
4   NaN
Name: one, dtype: float64

---

## Um jeito melhor

### 1ª abordagem

In [4]:
def coalesce(self, columns):
    result = pd.Series(df[columns[0]], name='coalesced')
    for c in columns[1:]:
        result = result.fillna(df[c])
    return result

Usando **_monkey patch_** é possível modificar uma classe em tempo de execução para adicionar um novo comportamento (ou destruir coisas!).

In [5]:
pd.DataFrame.coalesce = coalesce

In [6]:
df.coalesce(['one', 'two', 'three'])

0     1
1     2
2     3
3     1
4   NaN
Name: coalesced, dtype: float64

### Uma abordagem mais "Pythônica"?

In [7]:
def coalesce(self, columns):
    from functools import reduce
    return reduce(lambda series, col: series.fillna(self[col]),
                  columns[1:],
                  pd.Series(df[columns[0]], name='coalesced'))

Usando **_monkey patch_** de novo pra atualizar a função na classe `pandas.DataFrame`

In [8]:
pd.DataFrame.coalesce = coalesce

In [9]:
df.coalesce(['one', 'two', 'three'])

0     1
1     2
2     3
3     1
4   NaN
Name: coalesced, dtype: float64

In [10]:
from IPython.display import Image
Image(url='tada.png')

# Contato

braunmagrin@gmail.com